In [1]:
import os
import cv2
import csv
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
import torchvision
from torch import nn,optim
from torch.nn import functional as F
from torchvision import transforms 
from torchvision import models as m
from torch.utils.data import DataLoader
import torch
from PIL import Image
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import time
import datetime
import gc  #garbage collector
import json
from timm.models.vision_transformer import VisionTransformer
from torchinfo import summary
import cv2
# 导入对应的模型
# from model import swin_tiny_patch4_window7_224 as create_model

In [2]:
torch.cuda.manual_seed(1412)
torch.cuda.manual_seed_all(1412)
torch.manual_seed(420)

In [3]:
vgg16_bn_ =m.vgg16_bn() 
resnet18_ = m.resnet18()

In [4]:
class BasicConv2d_two(nn.Module):
    def __init__(self,in_,out_,f1,**kwargs):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_,out_, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
          )
            
            
    def forward(self,x):
        x = self.conv1(x)
        return x   

In [6]:
class MyNet3(nn.Module):
    def __init__(self,in_features=3):
        super().__init__()
        self.conv1 = nn.Sequential(BasicConv2d_two( in_features, 32, 32,kernel_size=3,stride=1,padding=1))
        self.conv2 = nn.Sequential(BasicConv2d_two( 32, 64, 64,kernel_size=3,stride=1,padding=1))
        self.block1=resnet18_.layer2
        self.block2=resnet18_.layer3
        self.avgpool=vgg16_bn_.avgpool
        self.classifer=nn.Sequential( nn.Linear(12544, 256, bias=True)
                                     ,nn.BatchNorm1d(256)
                                     ,nn.Dropout(p=0.5)
                                     ,nn.Linear(256,128, bias=True)
                                     ,nn.BatchNorm1d(128)
                                     ,nn.Dropout(p=0.5)
                                    )
        self.fc = nn.Linear(128,5)

        
    def forward(self,x):
        x = self.conv1(x)
        x= self.conv2(x)
        x= self.block1(x)
        x = self.block2(x)
        x = self.avgpool(x)
        x=x.view(-1,12544)
        x = self.classifer(x)
#         x = self.fc(x)
        return x  

In [15]:
def get_label_from_filename(filename):
    # 从文件名中提取标签
    label = filename.split('_')[-1].split('.')[0]
    return int(label)

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 创建模型实例
model = MyNet3()
model.load_state_dict(torch.load("/home/ubuntu/fy/外部验证/aptos_data/MyNet3_0.001_50_aptos_1.pt"))
model.eval()

folder_path = "/home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label"
out_csv_path="/home/ubuntu/fy/外部验证/aptos_data"
file_path_list = []
features = []
labels = []

for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    file_path_list.append(file_path)
    print("文件：", file_path)

    # 添加标签
    label = get_label_from_filename(file)
    labels.append(label)

    # 提取特征并保存为CSV
for path in file_path_list:
    # 读取文件
    image = Image.open(path)
    # 预处理图像
    processed_image = preprocess(image)
    processed_image = torch.unsqueeze(processed_image, dim=0)
    # 将图像输入到模型中提取特征
    with torch.no_grad():
        feature = model(processed_image)

    # 将特征转换为NumPy数组并添加到特征列表中
    feature = feature.squeeze().numpy()
    features.append(feature)


    # 将特征和标签保存为CSV文件
    output_path = os.path.join(out_csv_path, 'output_test_aptos_2.csv')

    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for feature, label in zip(features, labels):
            row = list(feature) + [label]
            writer.writerow(row)

文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/b82d5f1f1145_2.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/521b5377a727_0.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/aa5ce75edcf5_2.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/11242a67122d_4.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/3128eb593012_2.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/14ee87d6cc42_2.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/a88f68b0b114_2.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/9e2058917304_0.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/b2ffa3e18559_3.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/9782c0489eca_1.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/53ddae6a619e_0.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/f7edc074f06b_2.tif
文件： /home/ubuntu/fy/外部验证/aptos_data/aptos_ori_green_label/c62585bd68fb_4.tif